<a href="https://colab.research.google.com/github/yagmurdenizlii/nlp/blob/main/nlp_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New Section

In [ ]:
!pip install torch
!pip install torchvision
!pip install nltk
! pip install glove-python-binary



In [ ]:
import random
from sklearn.model_selection import train_test_split
import csv
import numpy as np
import statistics
import os
import re
import pandas as pd
import torch
from tensorflow.python.ops.numpy_ops import np_config
from torch.utils.data import DataLoader, TensorDataset, random_split
import torch.nn.functional as F
import torch.nn as nn
import matplotlib.pyplot as plt
import re  
import pandas as pd 
from time import time 
from collections import defaultdict  

import spacy 
from gensim.models import Word2Vec

import multiprocessing

from gensim.models import Word2Vec
import nltk 
nltk.download('stopwords')
nltk.download ('punkt')
nltk.download('wordnet')

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer  
from nltk.stem import WordNetLemmatizer
nltk.download('omw-1.4')
from glove import Corpus, Glove
from sklearn.metrics.pairwise import cosine_similarity

import torchtext


In [ ]:
artist_names = [    'Beyonce', 
            'BillieEilish', 
            'CharliePuth',
            'ColdPlay', 
            'Drake', 
            'DuaLipa',
            'EdSheeran',
            'JustinBieber',
            'KatyPerry',
            'LadyGaga',
            'Maroon5',
            'NickiMinaj',
            'Rihanna',
            'SelenaGomez',
            'TaylorSwift',
            ]

    
list_with_spaces = ['Beyonce', 
                    'Billie Eilish', 
                    'Charlie Puth',
                    'ColdPlay', 
                    'Drake', 
                    'Dua Lipa',
                    'Ed Sheeran',
                    'Justin Bieber',
                    'Katy Perry',
                    'Lady Gaga',
                    'Maroon 5',
                    'Nicki Minaj',
                    'Rihanna',
                    'Selena Gomez',
                    'Taylor Swift',
                    'beyoncé',
                    'verse',
                    'chris martin',
                    'sean paul',
                    'justin timberlake',
                    'beyoncè'
                    ]


In [ ]:

list_of_files = [artist + '.csv' for artist in artist_names]

all_lyrics = []

for file_path in list_of_files:
    df = pd.read_csv(file_path)
    df = df.dropna()

    text_list = df['Lyric'].values.tolist() 

    new_all_lyrics = []

    for line in text_list:
        new_line = line.replace("'", '')
        for name in list_with_spaces:
            new_line = new_line.replace(name.lower(), '')
        new_all_lyrics.append(new_line)

    for row in new_all_lyrics:
        if type(row) is str:
            row_list = row.split()
            nb_words = len(row_list)
            if nb_words > 200:
                nb_chunks = nb_words // 200 + 1
                arr = np.array_split(row_list, nb_chunks)

                for array in arr:
                    all_lyrics.append(' '.join(array))

            elif nb_words > 20:
                all_lyrics.append(row)

print('fin')       

In [ ]:
#gets the tokenized list of words, returns the list after removing the stopwords
def remove_stopwords(all):
  all_words_final = []
  for sentence in all:
    sentence_final = []
    for word in sentence:
      if word not in stopwords.words('english'):
        sentence_final.append(word)
    all_words_final.append(sentence_final)

  return all_words_final

In [ ]:
def my_lemmatizer(all):
  wordnet_lemmatizer = WordNetLemmatizer() 

  lemmatized = [] 

  for sentence in all: 
    lemmatized_words = []
    for word in sentence:
      lemmatized_words.append(wordnet_lemmatizer.lemmatize(word)) 
    lemmatized.append(lemmatized_words)

  return lemmatized

In [ ]:
#divide into two groups
training_data, test_data = train_test_split(all_lyrics, test_size = 0.8, random_state=42)


# New Section
Word2Vec

In [ ]:
#Tokenize
all_words = [nltk.word_tokenize(sent) for sent in training_data]

In [ ]:
#Remove stopwords
all_words_r = remove_stopwords(all_words)

In [ ]:
word2vec = Word2Vec(all_words_r, min_count=1, hs=1, negative=0)

In [ ]:
#training
word2vec.train(all_words_r, total_examples = word2vec.corpus_count, epochs=10, report_delay=1)


In [ ]:
test_tokens = [nltk.word_tokenize(sent) for sent in test_data]

scores = word2vec.score(test_tokens, len(test_tokens))


In [ ]:
pp = []
i = 0

for t in test_tokens:
  pp.append(2 ** (scores[i] / len(t)))
  i += 1

print(pp)

average = np.sum(pp) / len(pp)
print(average)


# New Section
Word2Vec 2

In [ ]:
w2v_model = Word2Vec(min_count=20,
                     window=2,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=3)

In [ ]:
w2v_model.build_vocab(all_words_r, progress_per=10000)

In [ ]:
w2v_model.train(all_words_r, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

In [ ]:
similar_words = w2v_model.wv.most_similar('love')
w2v_model.wv.most_similar(positive=["love", "amazing"], negative=["hate"], topn=3)

In [ ]:
w2v_model

# New Section

In [ ]:
#training_data, test_data

test_tokens = [nltk.word_tokenize(sent) for sent in test_data]
test_tokens = my_lemmatizer(remove_stopwords(test_tokens))



In [ ]:

unigram = {}
for song in all_words_r:
  for f in song:
      try:
          unigram[f] += 1
      except KeyError:
          unigram[f] = 1
          continue

print(unigram)

N = float(sum(unigram.values()))
for word in unigram:
    unigram[word] = unigram[word]/N

print(unigram)


# New Section

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
import string

In [ ]:

filename = "alice.txt"
text = open(filename, 'r', encoding='utf-8').read()
text = text.lower()
text = text.translate(str.maketrans('', '', string.punctuation))



In [ ]:
chars = sorted(list(set(text)))

#index = chars.index('z') + 1
#chars = chars[:index]

char_to_int = dict((c, i) for i, c in enumerate(chars))

n_chars = len(text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)



In [ ]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = text[i:i + seq_length]
	seq_out = text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

In [ ]:
...
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = to_categorical(dataY)

In [ ]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [ ]:
model.fit(X, y, epochs=50, batch_size=64, callbacks=callbacks_list)

In [ ]:
filename = 'weights-improvement-50-1.1209.hdf5'
pre_model = load_model(filename)
pre_model.compile(loss='categorical_crossentropy', optimizer='adam')

int_to_char = dict((i, c) for i, c in enumerate(chars))

In [ ]:
# pick a random seed
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(200):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = pre_model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_char[index]
    print(result, end ="")
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

# New Section

In [ ]:
import string

filename = "alice.txt"
text = open(filename, 'r', encoding='utf-8').read()
text = text.lower()
text = text.translate(str.maketrans('', '', string.punctuation))
text = text.split()

words = sorted(list(set(text)))

print(words)

#index = chars.index('z') + 1
#chars = chars[:index]

word_to_int = dict((c, i) for i, c in enumerate(words))

n_words = len(text)
n_vocab = len(words)
print ("Total words: ", n_words)
print ("Total Vocab: ", n_vocab)

In [ ]:
seq_length = 50
dataX = []
dataY = []
for i in range(0, n_words - seq_length, 1):
	seq_in = text[i:i + seq_length]
	seq_out = text[i + seq_length]
	dataX.append([word_to_int[word] for word in seq_in])
	dataY.append(word_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

In [ ]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = to_categorical(dataY)

# define the LSTM model
w_model = Sequential()
w_model.add(LSTM(512, input_shape=(X.shape[1], X.shape[2])))
w_model.add(Dropout(0.2))
w_model.add(Dense(y.shape[1], activation='softmax'))
w_model.compile(loss='categorical_crossentropy', optimizer='adam')


In [ ]:
filepath="aalice-weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [ ]:
w_model.fit(X, y, epochs=3, batch_size=128, callbacks=callbacks_list)

In [ ]:
filename = 'aalice-weights-improvement-14-6.0146.hdf5'
pre_model_alice = load_model(filename)
pre_model_alice.compile(loss='categorical_crossentropy', optimizer='adam')

int_to_word = dict((i, c) for i, c in enumerate(words))

In [ ]:
# pick a random seed
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ' '.join([int_to_word[value] for value in pattern]), "\"")
# generate characters
for i in range(100):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = pre_model_alice.predict(x, verbose=0)
    index = np.argmax(prediction)
    for i in range(random.randint(0, 2)):
      prediction = np.delete(prediction, index)
      index = np.argmax(prediction)
    result = int_to_word[index]
    print(result, end =" ")
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")